In [1]:
import os
import sys
import scipy.io as spio
import numpy as np
import plotly.graph_objects as pgo
import re

sys.path.append("../")
from helpers.get_density import get_density
from helpers.get_spins import get_spins
from helpers.downscale import downscale_3d
from plot_multiple_3Ds import plot_multiple_3Ds
from loader import get_snap_paths, load_Psi
from helpers.grids import get_kw_square_nonzero_grid, get_radius_grid, get_space_grid
from multiprocessing import Pool

In [2]:
box_N = 96
box_L = 400.0
box_dx = box_L / box_N

In [4]:
lambdas = [0]
runs = [1]
dtos = [1, 2, 4]
to_loads = []
for ld in lambdas:
	for run in runs:
		sr = {}
		for dto in dtos:
			sim_path = f"../out_remote/2022-09-11/4-solitons-attractive,lambda={ld},run={run},dto={dto}/"
			snap_paths = get_snap_paths(sim_path, box_L, box_N)
			for (itr, time, path) in snap_paths:
				to_loads.append((ld, run, dto, itr, time, path))


In [ ]:
def load_one(x):
	(ld, run, dto, itr, time, path) = x
	return (ld, run, dto, itr, time, load_Psi(path))

with Pool(32) as p:
 	loadeds = p.map(load_one, to_loads)

In [21]:

def test_for(test_ld, test_run):
	snaps = {dto: [] for dto in dtos}
	for (ld, run, dto, itr, time, Psi3) in sorted(loadeds):
		if (test_ld == ld and test_run == run):
			snaps[dto].append((time, Psi3))
	
	rhos = {dto: [(t,get_density(Psi3)) for (t,Psi3) in Psi3s] for (dto, Psi3s) in snaps.items()}
	dt1 = (2 / np.pi) * (box_dx)**2
	for ((t1, rho1), (t2, rho2), (t4, rho4)) in zip(rhos[1], rhos[2], rhos[4]):
		assert(abs(t1-t2) < 20 and abs(t2-t4) < 20)
		v = np.abs(rho1 - rho2) / np.abs(rho2 - rho4)
		q = np.log2(v)
		# print(np.percentile(q, 1), np.percentile(q, 10), np.percentile(q, 50), np.percentile(q, 90), np.percentile(q, 99))
		print(np.average(q))
		
test_for(0, 1)

1.6330975249729194
1.3106160641610194
1.243665927566465
1.1812393592189945
1.20948280941929
1.7124344096294581
1.9579610180456462
2.026445679818972
2.0011858902134483
1.9414893545285992
1.795437431995821
1.4307161072558676
1.178919865648366
1.1335701851765816
1.119376422181645
1.0364085083790793
0.9534805926332195
0.8187094333808738
0.7667124751456517
0.7151571299524455
0.6859824065560081
0.6510991473529893
0.6240358212192276
0.565836269046283
0.5300357596400507
0.5272846101575086
0.49939507778467357
0.5007423015575136
0.47058571242133246
0.46542679002617376
0.41140690722072504
0.3660870101874037
0.35841381003627815
0.3174712284263993
0.3223749763952974
0.2843759095856767
0.2933745710401042
0.29888643874965687
0.24886478595629988
0.2669064852486772
0.23871798009639386
0.2460767925244768
0.2117939401327799
0.2183613310324094
0.2013141519059096
0.24095583551507233
0.20920404986771443
0.22436107207839665
0.21043260510640385
0.19605440585449593
0.22282819115487734
0.17783801881126587
0.181